In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os

from tensorflow.contrib.framework import arg_scope
from tensorflow.contrib.layers import batch_norm, flatten

In [2]:
#init weight and placeholder

num_classes = 1
height = 256
width = 256

x = tf.placeholder(tf.float32, [None, height, width,num_classes],name="x")
y = tf.placeholder(tf.float32, [None, height, width],name="y")

p_keep_conv = tf.placeholder("float",name="p_keep_conv")
is_training = tf.placeholder(tf.bool,name="is_training")

ksize = 5
fsize = 64
initstdev = 0.01
initfun = tf.random_normal_initializer(mean=0.0, stddev=initstdev)

weights = {
    'conv1_1': tf.get_variable("conv1_1", shape = [3, 3, num_classes, 64], initializer = initfun) ,
    'conv1_2': tf.get_variable("conv1_2", shape = [3, 3, 64, 64], initializer = initfun) ,
    'conv1_3': tf.get_variable("conv1_3", shape = [3, 3, 128, 64], initializer = initfun) ,
    'conv1_4': tf.get_variable("conv1_4", shape = [3, 3, 64, 64], initializer = initfun) ,
    
    'conv2_1': tf.get_variable("conv2_1", shape = [3, 3, 64, 128], initializer = initfun) ,
    'conv2_2': tf.get_variable("conv2_2", shape = [3, 3, 128, 128], initializer = initfun) ,
    'conv2_3': tf.get_variable("conv2_3", shape = [3, 3, 256, 128], initializer = initfun) ,
    'conv2_4': tf.get_variable("conv2_4", shape = [3, 3, 128, 128], initializer = initfun) ,
    'conv2_de': tf.get_variable("conv2_de", shape = [3, 3, 64, 128], initializer = initfun) ,
    
    'conv3_1': tf.get_variable("conv3_1", shape = [3, 3, 128, 256], initializer = initfun) ,
    'conv3_2': tf.get_variable("conv3_2", shape = [3, 3, 256, 256], initializer = initfun) ,
    'conv3_3': tf.get_variable("conv3_3", shape = [3, 3, 512, 256], initializer = initfun) ,
    'conv3_4': tf.get_variable("conv3_4", shape = [3, 3, 256, 256], initializer = initfun) ,
    'conv3_de': tf.get_variable("conv3_de", shape = [3, 3, 128, 256], initializer = initfun) ,
    
    'conv4_1': tf.get_variable("conv4_1", shape = [3, 3, 256, 512], initializer = initfun) ,
    'conv4_2': tf.get_variable("conv4_2", shape = [3, 3, 512, 512], initializer = initfun) ,
    'conv4_3': tf.get_variable("conv4_3", shape = [3, 3, 1024, 512], initializer = initfun) ,
    'conv4_4': tf.get_variable("conv4_4", shape = [3, 3, 512, 512], initializer = initfun) ,
    'conv4_de': tf.get_variable("conv4_de", shape = [3, 3, 256, 512], initializer = initfun) ,
    
    'conv5_1': tf.get_variable("conv5_1", shape = [3, 3, 512, 1024], initializer = initfun) ,
    'conv5_2': tf.get_variable("conv5_2", shape = [3, 3, 1024, 1024], initializer = initfun) ,
    'conv5_de': tf.get_variable("conv5_de", shape = [3, 3, 512, 1024], initializer = initfun) ,
    
    'dense_inner_prod': tf.get_variable("dense_inner_prod", shape= [1, 1, 64, num_classes]
                                       , initializer = initfun)
}
biases = {
    'b1_1': tf.get_variable("b1_1", shape = [64], initializer = tf.constant_initializer(value=0.0)),
    'b1_2': tf.get_variable("b1_2", shape = [64], initializer = tf.constant_initializer(value=0.0)),
    'b1_3': tf.get_variable("b1_3", shape = [64], initializer = tf.constant_initializer(value=0.0)),
    'b1_4': tf.get_variable("b1_4", shape = [64], initializer = tf.constant_initializer(value=0.0)),
    
    'b2_1': tf.get_variable("b2_1", shape = [128], initializer = tf.constant_initializer(value=0.0)),
    'b2_2': tf.get_variable("b2_2", shape = [128], initializer = tf.constant_initializer(value=0.0)),
    'b2_3': tf.get_variable("b2_3", shape = [128], initializer = tf.constant_initializer(value=0.0)),
    'b2_4': tf.get_variable("b2_4", shape = [128], initializer = tf.constant_initializer(value=0.0)),
    
    'b3_1': tf.get_variable("b3_1", shape = [256], initializer = tf.constant_initializer(value=0.0)),
    'b3_2': tf.get_variable("b3_2", shape = [256], initializer = tf.constant_initializer(value=0.0)),
    'b3_3': tf.get_variable("b3_3", shape = [256], initializer = tf.constant_initializer(value=0.0)),
    'b3_4': tf.get_variable("b3_4", shape = [256], initializer = tf.constant_initializer(value=0.0)),
    
    'b4_1': tf.get_variable("b4_1", shape = [512], initializer = tf.constant_initializer(value=0.0)),
    'b4_2': tf.get_variable("b4_2", shape = [512], initializer = tf.constant_initializer(value=0.0)),
    'b4_3': tf.get_variable("b4_3", shape = [512], initializer = tf.constant_initializer(value=0.0)),
    'b4_4': tf.get_variable("b4_4", shape = [512], initializer = tf.constant_initializer(value=0.0)),
    
    'b5_1': tf.get_variable("b5_1", shape = [1024], initializer = tf.constant_initializer(value=0.0)),
    'b5_2': tf.get_variable("b5_2", shape = [1024], initializer = tf.constant_initializer(value=0.0)),
    
    'b5': tf.get_variable("bd4", shape = [256], initializer = tf.constant_initializer(value=0.0)),
    'b6': tf.get_variable("bd3", shape = [256], initializer = tf.constant_initializer(value=0.0)),
    'b7': tf.get_variable("bd2", shape = [256], initializer = tf.constant_initializer(value=0.0)),
    'bdp': tf.get_variable("bd1", shape = [256], initializer = tf.constant_initializer(value=0.0))
}

Instructions for updating:
Colocations handled automatically by placer.


In [3]:

# U-net Model

def Unpooling(inputOrg, size, mask=None):
    m = size[0]
    h = size[1]
    w = size[2]
    c = size[3]
    input = tf.transpose(inputOrg, [0, 3, 1, 2])
    x = tf.reshape(input, [-1, 1])
    k = np.float32(np.array([1.0, 1.0]).reshape([1,-1]))
    output = tf.matmul(x, k)
    output = tf.reshape(output,[-1, c, h, w * 2])
    xx = tf.transpose(output, [0, 1, 3, 2])
    xx = tf.reshape(xx,[-1, 1])
    output = tf.matmul(xx, k)
    output = tf.reshape(output, [-1, c, w * 2, h * 2])
    output = tf.transpose(output, [0, 3, 2, 1])
    outshape = tf.stack([m, h * 2, w * 2, c])
    if mask != None:
        dense_mask = tf.sparse_to_dense(mask, outshape, output, 0)
        return output, dense_mask
    else:
        return output
    
def Batch_Normalization(x, training, scope):
    with arg_scope([batch_norm],
                   scope=scope,
                   updates_collections=None,
                   decay=0.9,
                   center=True,
                   scale=True,
                   zero_debias_moving_mean=True) :
        return tf.cond(training,
                       lambda : batch_norm(inputs=x, is_training=training, reuse=None),
                       lambda : batch_norm(inputs=x, is_training=training, reuse=True))
    
    
def Model(_X, _W, _b, _keepprob,is_training):

    layer1 = tf.nn.conv2d(_X, _W['conv1_1'], strides=[1, 1, 1, 1], padding='SAME')
    layer1 = tf.nn.bias_add(layer1, _b['b1_1'])
           
    layer1 = Batch_Normalization(layer1, training=is_training, scope="bn_layer1_1")
    
    layer1 = tf.nn.relu(layer1)
    
    layer1 = tf.nn.conv2d(layer1, _W['conv1_2'], strides=[1, 1, 1, 1], padding='SAME')
    layer1 = tf.nn.bias_add(layer1, _b['b1_2'])
           
    layer1 = Batch_Normalization(layer1, training=is_training, scope="bn_layer1_2")
    
    layer1 = tf.nn.relu(layer1)
    
    #########################################################################################
    
    layer2 = tf.nn.max_pool(layer1,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    layer2 = tf.nn.conv2d(layer2, _W['conv2_1'], strides=[1, 1, 1, 1], padding='SAME')
    layer2 = tf.nn.bias_add(layer2, _b['b2_1'])
           
    layer2 = Batch_Normalization(layer2, training=is_training, scope="bn_layer2_1")
    
    layer2 = tf.nn.relu(layer2)
    
    layer2 = tf.nn.conv2d(layer2, _W['conv2_2'], strides=[1, 1, 1, 1], padding='SAME')
    layer2 = tf.nn.bias_add(layer2, _b['b2_2'])
           
    layer2 = Batch_Normalization(layer2, training=is_training, scope="bn_layer2_2")
    
    layer2 = tf.nn.relu(layer2)
    
    ##########################################################################################
    
    layer3 = tf.nn.max_pool(layer2,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    layer3 = tf.nn.conv2d(layer3, _W['conv3_1'], strides=[1, 1, 1, 1], padding='SAME')
    layer3 = tf.nn.bias_add(layer3, _b['b3_1'])
           
    layer3 = Batch_Normalization(layer3, training=is_training, scope="bn_layer3_1")
    
    layer3 = tf.nn.relu(layer3)
    
    layer3 = tf.nn.conv2d(layer3, _W['conv3_2'], strides=[1, 1, 1, 1], padding='SAME')
    layer3 = tf.nn.bias_add(layer3, _b['b3_2'])
           
    layer3 = Batch_Normalization(layer3, training=is_training, scope="bn_layer3_2")
    
    layer3 = tf.nn.relu(layer3)
    
    ##########################################################################################
    
    layer4 = tf.nn.max_pool(layer3,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    layer4 = tf.nn.conv2d(layer4, _W['conv4_1'], strides=[1, 1, 1, 1], padding='SAME')
    layer4 = tf.nn.bias_add(layer4, _b['b4_1'])
           
    layer4 = Batch_Normalization(layer4, training=is_training, scope="bn_layer4_1")
    
    layer4 = tf.nn.relu(layer4)
    
    layer4 = tf.nn.conv2d(layer4, _W['conv4_2'], strides=[1, 1, 1, 1], padding='SAME')
    layer4 = tf.nn.bias_add(layer4, _b['b4_2'])
           
    layer4 = Batch_Normalization(layer4, training=is_training, scope="bn_layer4_2")
    
    layer4 = tf.nn.relu(layer4)
    
    ##########################################################################################
    
    layer5 = tf.nn.max_pool(layer4,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    layer5 = tf.nn.conv2d(layer5, _W['conv5_1'], strides=[1, 1, 1, 1], padding='SAME')
    layer5 = tf.nn.bias_add(layer5, _b['b5_1'])
           
    layer5 = Batch_Normalization(layer5, training=is_training, scope="bn_layer5_1")
    
    layer5 = tf.nn.relu(layer5)
    
    layer5 = tf.nn.conv2d(layer5, _W['conv5_2'], strides=[1, 1, 1, 1], padding='SAME')
    layer5 = tf.nn.bias_add(layer5, _b['b5_2'])
           
    layer5 = Batch_Normalization(layer5, training=is_training, scope="bn_layer5_2")
    
    layer5 = tf.nn.relu(layer5)
    
    ##########################################################################################
    
    layer5 = tf.nn.conv2d_transpose(layer5, _W['conv5_de'],tf.stack([tf.shape(_X)[0],int(height/8),int(width/8),512]) , [1, 2, 2, 1], padding="SAME")
    
    layer4 = tf.concat([layer4,layer5],axis=3)
    
    layer4 = tf.nn.conv2d(layer4, _W['conv4_3'], strides=[1, 1, 1, 1], padding='SAME')
    layer4 = tf.nn.bias_add(layer4, _b['b4_3'])
           
    layer4 = Batch_Normalization(layer5, training=is_training, scope="bn_layer4_3")
    
    layer4 = tf.nn.relu(layer4)
    
    layer4 = tf.nn.conv2d(layer4, _W['conv4_4'], strides=[1, 1, 1, 1], padding='SAME')
    layer4 = tf.nn.bias_add(layer4, _b['b4_4'])
           
    layer4 = Batch_Normalization(layer5, training=is_training, scope="bn_layer4_4")
    
    layer4 = tf.nn.relu(layer4)
    
    ##########################################################################################
    
    layer4 = tf.nn.conv2d_transpose(layer4, _W['conv4_de'],tf.stack([tf.shape(_X)[0],int(height/4),int(width/4),256]) , [1, 2, 2, 1], padding="SAME")
    
    layer3 = tf.concat([layer3,layer4],axis=3)
    
    layer3 = tf.nn.conv2d(layer3, _W['conv3_3'], strides=[1, 1, 1, 1], padding='SAME')
    layer3 = tf.nn.bias_add(layer3, _b['b3_3'])
           
    layer3 = Batch_Normalization(layer3, training=is_training, scope="bn_layer3_3")
    
    layer3 = tf.nn.relu(layer3)
    
    layer3 = tf.nn.conv2d(layer3, _W['conv3_4'], strides=[1, 1, 1, 1], padding='SAME')
    layer3 = tf.nn.bias_add(layer3, _b['b3_4'])
           
    layer3 = Batch_Normalization(layer3, training=is_training, scope="bn_layer3_4")
    
    layer3 = tf.nn.relu(layer3)
    
    ##########################################################################################
    
    layer3 = tf.nn.conv2d_transpose(layer3, _W['conv3_de'],tf.stack([tf.shape(_X)[0],int(height/2),int(width/2),128]) , [1, 2, 2, 1], padding="SAME")
    
    layer2 = tf.concat([layer2,layer3],axis=3)
    
    layer2 = tf.nn.conv2d(layer2, _W['conv2_3'], strides=[1, 1, 1, 1], padding='SAME')
    layer2 = tf.nn.bias_add(layer2, _b['b2_3'])
           
    layer2 = Batch_Normalization(layer2, training=is_training, scope="bn_layer2_3")
    
    layer2 = tf.nn.relu(layer2)
    
    layer2 = tf.nn.conv2d(layer2, _W['conv2_4'], strides=[1, 1, 1, 1], padding='SAME')
    layer2 = tf.nn.bias_add(layer2, _b['b2_4'])
           
    layer2 = Batch_Normalization(layer2, training=is_training, scope="bn_layer2_4")
    
    layer2 = tf.nn.relu(layer2)
    
    ##########################################################################################
    
    layer2 = tf.nn.conv2d_transpose(layer2, _W['conv2_de'],tf.stack([tf.shape(_X)[0],height,width,64]) , [1, 2, 2, 1], padding="SAME")
    
    layer1 = tf.concat([layer1,layer2],axis=3)
    
    layer1 = tf.nn.conv2d(layer1, _W['conv1_3'], strides=[1, 1, 1, 1], padding='SAME')
    layer1 = tf.nn.bias_add(layer1, _b['b1_3'])
           
    layer1 = Batch_Normalization(layer1, training=is_training, scope="bn_layer1_3")
    
    layer1 = tf.nn.relu(layer1)
    
    layer1 = tf.nn.conv2d(layer1, _W['conv1_4'], strides=[1, 1, 1, 1], padding='SAME')
    layer1 = tf.nn.bias_add(layer1, _b['b1_4'])
           
    layer1 = Batch_Normalization(layer1, training=is_training, scope="bn_layer1_4")
    
    layer1 = tf.nn.relu(layer1)
    
    
    output = tf.nn.conv2d(layer1, _W['dense_inner_prod'], strides=[1, 1, 1, 1], padding='SAME')
    
    return tf.reshape(output,[-1,height,width])

pred = Model(x, weights, biases,p_keep_conv,is_training)

In [4]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
ckpt = tf.train.get_checkpoint_state('./model/disc_segmentation_model/')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/disc_segmentation_model/max.model


In [5]:

def crop_margin_PIL(image):
    im = image
    np_img = np.asarray(im)
    mean_pix = np.mean(np_img)
    pix = im.load()
    height, width = im.size

    c_x, c_y = (int(height / 2), int(width / 2))

    for y in range(c_y):
        if sum(pix[c_x, y]) > mean_pix:
            left = (c_x, y)
            break;

    for x in range(c_x):
        if sum(pix[x, c_y]) > mean_pix:
            up = (x, c_y)
            break;

    crop_img = im.crop((up[0], left[1], left[0], up[1]))

    diameter_height = up[1] - left[1]
    diameter_width = left[0] - up[0]

    crop_img = im.crop((up[0], left[1], left[0] + diameter_width, up[1] + diameter_height))

    return crop_img

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6]:
result = ""

img_dir = "/home/mediwhale/SEED/SEED_1/RETINA - SCES/SICC05/2009-02-09/CS50023/CS50023_KK00(POZ.002.JPG"

img_origin = Image.open(img_dir)
img_origin = crop_margin_PIL(img_origin)
img = img_origin.convert("L").resize((256,256))
img_arr = np.array(img)

img_arr = img_arr.reshape(1,height,width,1)
img_arr = img_arr/255.

p = sess.run([pred], feed_dict={x: img_arr,p_keep_conv:1.0,is_training:False})
p = np.array(p)
p = sigmoid(p)
p = p.reshape(height,width)

threshold = p.max() - 0.1



    

In [7]:
if (p > 0.2).sum() < 50 :
    result = "other"
else :
    top = (p[0:95,0:85] > threshold).sum() + (p[0:60,85:170] > threshold).sum() + (p[0:95,170:] > threshold).sum()
    center = (p[95:160,0:85] > threshold).sum() + (p[60:196,85:170] > threshold).sum() + (p[95:160,170:] > threshold).sum()
    bottom = (p[160:,0:85] > threshold).sum() + (p[196:,85:170] > threshold).sum() + (p[160:,170:] > threshold).sum()
    
    if top > center or bottom > center :
        result = "other"
    else :
        left = (p[80:176,:85]>threshold).sum()
        center = (p[80:176,85:170]>threshold).sum()
        right = (p[80:176,170:]>threshold).sum()
        
        l = np.argmax([left,center,right])
        
        if l == 0 :
            result = "macular_focus_left"
        elif l == 1 :
            result = "disc_focus"
        elif l == 2 :
            result = "macular_focus_right"

In [8]:
print(result)

disc_focus
